In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp point_forecast

# Point Forecasts

> Functions to find point forecasts using simulated values from the forecast distribution.

Forecast samples are the output from `dglm.forecast_marginal`, `dglm.forecast_path`, and most commonly, `analysis`. All of the functions in this module accept an array of forecast samples and produce a series of point forecasts, such as the forecast mean or median.

When using `analysis`, the samples are sequentially simulated from the forecast distribution at each specified time step. The forecast samples are placed in a 3 dimensional array, whose axes are *nsamps* $\times$ *forecast length* $\times$ *k*, where:
- **nsamps** is the number of samples drawn from the forecast distribution
- **forecast length** is the number of time steps between `forecast_start` and `forecast_end` in `analysis`
- **k** is the forecast horizon, or the number of steps that were forecast ahead

The point forecast will be calculated over *nsamps*, meaning that the output will be a 2 dimensional array of size *forecast length* $\times$ *k*.

More generally, all of the point forecasts are calculated from an array and assume that random samples are stored along the first dimension.

In [ ]:
#hide
#exporti
import numpy as np

In [ ]:
#export

# Optimal for MSE or mean squared error
def mean(samps):
    """
    Find the mean point forecasts.
    """
    return np.mean(samps, axis=0)

The mean point forecast is theoretically optimal for minimizing squared error loss, $L = \sum_i (y_i - f_i)^2$.

An example below demonstrates how to use the function:

In [ ]:
from pybats.shared import load_us_inflation
from pybats.analysis import analysis
import pandas as pd

data = load_us_inflation()

forecast_start = '2000-Q1'
forecast_end = '2013-Q4'

mod, samples = analysis(Y = data.Inflation.values[1:], X=None, family="normal",
                        k = 4, prior_length = 12,
                        forecast_start = forecast_start, forecast_end = forecast_end,
                        dates=data.Date,
                        ntrend = 2, deltrend=.99,
                        nsamps = 5000)


forecast = mean(samples)

beginning forecasting


In [ ]:
forecast.shape

(56, 4)

In [ ]:
start = data[data.Date == forecast_start].index[0]
end = data[data.Date == forecast_end].index[0] + 1
dates = pd.to_datetime(data[start:end].Date)


forecast = pd.DataFrame(forecast, index=dates)
forecast.columns = ['1-Step Ahead', '2-Step Ahead', '3-Step Ahead', '4-Step Ahead']
forecast.round(2).head()

,1-Step Ahead,2-Step Ahead,3-Step Ahead,4-Step Ahead
Date,,,,
2000-01-01,0.99,0.96,0.87,0.80
2000-04-01,1.00,0.94,0.88,0.81
2000-07-01,1.06,1.01,0.93,0.88
2000-10-01,1.07,1.03,0.96,0.97
2001-01-01,1.13,1.05,1.04,1.02


The forecasts are made on each date *before* seeing the observation. So the $1-$Step ahead forecast is for the date listed in that row. The $2-$Step Ahead forecast is project the mean for the date listed in the next row, and so on.

This view allows you to easily see the forecasts and how the model projects into the future. Looking across the first row, it is clear that the model has a negative local slope, because the forecasts generally decrease as they go further into the future.

In [ ]:
#export

# Optimal for MAD or absolute deviation
def median(samps):
    """
    Find the median point forecasts.
    """
    return np.median(samps, axis=0)

The median point forecast is theoretically optimal for minimizing absolute deviation loss, $L = \sum_i |y_i - f_i|$.

In [ ]:
forecast = median(samples)

forecast = pd.DataFrame(forecast, index=dates)
forecast.columns = ['1-Step Ahead', '2-Step Ahead', '3-Step Ahead', '4-Step Ahead']
forecast.round(2).head()

,1-Step Ahead,2-Step Ahead,3-Step Ahead,4-Step Ahead
Date,,,,
2000-01-01,0.97,0.96,0.87,0.80
2000-04-01,1.02,0.96,0.90,0.82
2000-07-01,1.04,1.00,0.93,0.88
2000-10-01,1.06,1.02,0.96,0.99
2001-01-01,1.13,1.05,1.05,1.02


In [ ]:
#exporti

# Utility function
def weighted_quantile(samp, weights, quantile=0.5):
    order = np.argsort(samp)
    ord_samp = samp[order]
    ord_weights = weights[order]
    lower = ord_samp[np.max(np.where(np.cumsum(ord_weights) < quantile))]
    upper = ord_samp[np.min(np.where(np.cumsum(ord_weights) > quantile))]
    return (upper + lower) / 2

In [ ]:
#export

# Optimal for APE. Always less than the median. Ignores samples that are 0.
def m_one_median(samps):
    """
    Find the (-1)-median point forecasts.
    """
    def m_one_median(samp):
        nz = samp.nonzero()[0]
        weights = 1/samp[nz]
        norm = np.sum(weights)
        weights = weights/norm
        if len(nz) < 5:
            print('Less than 5 non-zero samples')
        return weighted_quantile(samp[nz], weights)

    forecast = np.apply_along_axis(m_one_median, 0, samps)

    return forecast

The $\left(-1\right)-$median is theoretically optimal for minimizing absolute percent error, $L=\sum_i |y_i - f_i|/y_i$. It is always lower than the median, and is only defined for non-zero $y$.

We'll use a new example, because the $(-1)-$median can produce strange results when $y_i$ is close to $0$.

In [ ]:
from pybats.shared import load_sales_example

data = load_sales_example()             

Y = data['Sales'].values
X = data['Advertising'].values

k = 4                                               
forecast_start = 15                                
forecast_end = 30                               

mod, samples = analysis(Y, X, family="poisson",
                        forecast_start=forecast_start, forecast_end=forecast_end,
                        k=k, prior_length=6,
                        rho=.5, deltrend=0.95, delregn=0.95)

beginning forecasting


In [ ]:
forecast = m_one_median(samples)

forecast = pd.DataFrame(forecast)
forecast.columns = ['1-Step Ahead', '2-Step Ahead', '3-Step Ahead', '4-Step Ahead']
forecast.round(2).head()

,1-Step Ahead,2-Step Ahead,3-Step Ahead,4-Step Ahead
0,71.5,73.5,47.5,52.0
1,68.0,44.0,48.0,39.0
2,37.0,39.0,34.0,29.5
3,40.0,35.0,30.0,22.0
4,31.0,27.0,20.0,19.0


In [ ]:
#exporti

# Here we get the joint one_median, where the rows are forecast samples
# Assume that the forecast is 'joint' across the second dimension
# This is optimal for the WAPE loss, where the denominator in the WAPE score is the sum over the second dimension
# If the forecast samples are from a standard analysis function, that will be the sum over all forecast dates
def joint_m_one_median(samps):

    def joint_m_one_median(samp):
        rows, cols = samp.shape
        # Remove rows that are all zero
        rowsums = np.sum(samp, axis=1)
        psamp = samp[rowsums.nonzero()[0], :]
        rowsums = rowsums[rowsums.nonzero()[0]]

        # Weight each joint sample (i.e. row) by the inverse of its sum
        weights = 1 / rowsums
        norm = np.sum(weights)
        weights = weights / norm

        # Get the -1 median for each column using these joint weights
        forecast = np.zeros(cols)
        for c in range(cols):
            forecast[c] = weighted_quantile(psamp[:, c], weights)

        return forecast

    if samps.ndim == 2:
        return joint_m_one_median(samps)
    elif samps.ndim == 3:
        return np.array(list(map(joint_m_one_median, samps.transpose([1,0,2]))))

In [ ]:
#exporti

# For the constrained point forecasts
# F is a vector of constraints for the totals across the 3rd dimension of 'samps'
# Expected dimensions are: nsamps x time x (forecast horizon or items)
def constrained_mean(samps, F):
    means = np.mean(samps, axis=0)
    n = means.shape[1]
    diff = (F - np.sum(means, axis=1))/n
    return means + diff.reshape(-1,1)

In [ ]:
#exporti

def constrained_median(samps, F):
    if samps.ndim == 2:
        samps = np.expand_dims(samps, axis=1)

    # Initialize values
    forecast = median(samps)
    times = forecast.shape[0]
    lambd = np.zeros(times)

    # Iterate until a solution is found for each lambda
    tol = 1
    eps = 1E-2
    max_shift = 5E-2
    iter = 0
    max_iter = 50
    diff = F - np.sum(forecast, axis=1)
    test = np.abs(diff) > tol

    while np.any(test):
        shift = np.abs(eps*diff)
        shift[shift > max_shift] = max_shift
        lambd = lambd + np.sign(diff)*shift
        percentiles = 100*(1+lambd)/2
        for idx, p in enumerate(percentiles):
            if test[idx]:
                forecast[idx,:] = np.percentile(samps[:,idx,:], p, axis=0, interpolation='nearest')
        diff = F - np.sum(forecast, axis=1)
        test = np.abs(diff) > tol
        iter += 1
        if iter > max_iter:
           break
    return forecast

In [ ]:
#exporti

def constrained_joint_m_one_median(samps, F):


    def constrained_joint_m_one_median(samp, F):
        #if samp.ndim == 2:
        #    samp = np.expand_dims(samp, axis=1)

        # Remove joint samples that are all 0
        rowsums = np.sum(samp, axis=1)
        nz = rowsums.nonzero()[0]
        samp = samp[nz,:]
        rowsums = rowsums[nz]
        # Find weights
        weights = 1 / rowsums
        norm = np.sum(weights)
        weights = weights / norm

        # Initialize value
        forecast = joint_m_one_median(samp).reshape(1,-1)
        times = forecast.shape[0]
        lambd = np.zeros(times)

        # Iterate until a solution is found for each lambda
        tol = 1
        eps = 1E-2
        max_shift = 5E-2
        iter = 0
        max_iter = 50
        diff = F - np.sum(forecast)
        test = np.abs(diff) > tol

        while np.any(test):
            shift = np.abs(eps * diff)
            if shift > max_shift:
                shift = max_shift
            lambd = lambd + np.sign(diff) * shift
            percentile = 100 * (1 + lambd) / 2
            forecast = np.array(list(map(lambda s: weighted_quantile(s, weights, percentile/100),
                                                 samp.T)))
            diff = F - np.sum(forecast)
            test = np.abs(diff) > tol
            iter += 1
            if iter > max_iter:
                break
        return forecast.reshape(1,-1)

    if samps.ndim == 2:
        samps = np.expand_dims(samps, axis=1)

    return np.array(list(map(lambda samp, F: constrained_joint_m_one_median(samp, F),
                             samps.transpose([1, 0, 2]),
                             F)))[:,0,:]

In [ ]:
#exporti

# Optimal for ZAPE. Always less than the (-1)-median.
def zape_point_estimate(samps):
    """
    Return the optimal point forecast for ZAPE loss, given samples from the analysis function.

    This forecast is theoretically optimal for minimizing ZAPE loss, which is defined as:

    .. math:: ZAPE(y, f) = \\frac{1}{n} \sum_{i=1:n} I(y_i = 0) * f_i + I(y_i = 1) * |y_i-f_i| / y_i

    :param samps: Forecast samples, returned from the analysis function. Will have 3-dimensions (nsamps * time * forecast horizon)
    :return: Array of (-1)-median forecasts. Will have dimension (time * forecast horizon)
    """
    def est_c_hat(samp):
        nz = samp.nonzero()[0]
        weights = 1/samp[nz]
        c_hat = 1 / (1/len(nz) * np.sum(weights))
        return c_hat

    def zape_point_est(samp):
        nz = samp.nonzero()[0]
        pi_0 = len(nz) / len(samp) # probability of 0
        weights = 1 / samp[nz]
        norm = np.sum(weights)
        weights = weights / norm
        c_hat = est_c_hat(samp)
        quantile = (1 - c_hat*pi_0)/2

        return weighted_quantile(samp[nz], weights, quantile)

    forecast = np.apply_along_axis(m_one_median, 0, samps)

    return forecast

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_dglm.ipynb.
Converted 01_update.ipynb.
Converted 02_forecast.ipynb.
Converted 03_define_models.ipynb.
Converted 04_seasonal.ipynb.
Converted 05_analysis.ipynb.
Converted 06_conjugates.ipynb.
Converted 07_point_forecast.ipynb.
Converted 08_loss_functions.ipynb.
Converted 09_plot.ipynb.
Converted 10_shared.ipynb.
Converted 11_dcmm.ipynb.
Converted 12_dbcm.ipynb.
Converted 13_latent_factor.ipynb.
Converted 14_latent_factor_fxns.ipynb.
Converted index.ipynb.
